<a href="https://colab.research.google.com/github/shoaibakhtar20/Spark/blob/master/SapientAssessment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz


!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()

In [38]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("SapientAssessment").config("spark.some.config.option", "some-value").getOrCreate()

In [57]:
clickstream_raw = spark.read.csv("/content/drive/My Drive/UdemyHandsOn/Sapient_Clickstrea.csv", header='true')
clickstream_raw.show()

+--------------------+------+
|           timestamp|userid|
+--------------------+------+
|2018-01-01T11:00:00Z|    u1|
|2018-01-01T12:00:00Z|    u1|
|2018-01-01T11:00:00Z|    u2|
|2018-01-02T11:00:00Z|    u2|
|2018-01-01T12:15:00Z|    u1|
+--------------------+------+



In [65]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import unix_timestamp
import pyspark.sql.functions as func
from pyspark.sql.functions import lit
import dateutil.parser

In [ ]:


inactivity_time = 30
total_time = 2*60*60

window_user = Window.partitionBy("userid").orderBy("timestamp")
clickstream_with_window = clickstream_raw.withColumn("time_diff", unix_timestamp('timestamp') - func.lag(unix_timestamp('timestamp')).over(window_user)).withColumn("time_diff", when((row_number.over(window_user) == 1 ) or (col("time_diff") > inactivity_time), '').otherwise($"time_diff") )
clickstream_with_window.show()